In [11]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [12]:
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [13]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [14]:
pages = np.arange(1, 1001, 50)

In [15]:
for page in pages: 

  page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)

  soup = BeautifulSoup(page.text, 'html.parser')
  movie_div = soup.find_all('div', class_='lister-item mode-advanced')
  
  sleep(randint(2,10))

  for container in movie_div:

        name = container.h3.a.text
        titles.append(name)
        
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)

        runtime = container.p.find('span', class_='runtime') if container.p.find('span', class_='runtime') else ''
        time.append(runtime)

        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else ''
        metascores.append(m_score)

        nv = container.find_all('span', attrs={'name': 'nv'})
        
        vote = nv[0].text
        votes.append(vote)
        
        grosses = nv[1].text if len(nv) > 1 else ''
        us_gross.append(grosses)

In [16]:
movies = pd.DataFrame({
'movie': titles,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
'timeMin': time
})

In [17]:
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)

movies.loc[:, 'year'] = movies['year'].str[-5:-1].astype(int)

movies['timeMin'] = movies['timeMin'].astype(str)
movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)

movies['metascore'] = movies['metascore'].str.extract('(\d+)')
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

In [18]:
# to see your dataframe
print(movies)

                            movie  year  imdb  metascore   votes  \
0    Zack Snyder's Justice League  2021   8.3       54.0  191400   
1                  Sound of Metal  2019   7.8       82.0   46613   
2     Judas and the Black Messiah  2021   7.6       85.0   27677   
3      The Trial of the Chicago 7  2020   7.8       77.0  115492   
4               Avengers: Endgame  2019   8.4       78.0  840703   
..                            ...   ...   ...        ...     ...   
995           Munna Bhai M.B.B.S.  2003   8.1        NaN   74474   
996                    Kai Po Che  2013   7.7       40.0   32942   
997                     Vizontele  2001   8.0        NaN   33921   
998                    The Breath  2009   8.0        NaN   32030   
999          Lage Raho Munna Bhai  2006   8.1        NaN   43381   

     us_grossMillions  timeMin  
0                 NaN      242  
1                 NaN      120  
2                 NaN      126  
3                 NaN      129  
4              858

In [19]:
# to see the datatypes of your columns
print(movies.dtypes)

movie                object
year                  int32
imdb                float64
metascore           float64
votes                 int32
us_grossMillions    float64
timeMin               int32
dtype: object


In [20]:
# to see where you're missing data and how much data is missing 
print(movies.isnull().sum())

movie                 0
year                  0
imdb                  0
metascore           157
votes                 0
us_grossMillions    169
timeMin               0
dtype: int64


In [21]:
# to move all your scraped data to a CSV file
movies.to_csv('movies.csv')